### ✍️ Required libraries & settings

In [ ]:
import sys
sys.path.insert(1, '/home/sbanik@quansight.com/demo-dashboards')

import warnings
warnings.filterwarnings('ignore')

from intake_utils import (
    catalog_init,
    list_catalog,
    view_catalog)

import numpy as np
import panel as pn
import geoviews as gv
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts,dim
from bokeh.models import HoverTool
hv.extension('bokeh')

### 👓 About dataset

- **Title :** Austin trees data
- **Brief description :** This dataset shows point locations of public trees inventoried by the City of Austin as of March 13th, 2020. Data is compiled from various sources: Development Services Department's Tree Division, AISD, Parks and Recreation Department, and Public Works Department's downtown tree inventory (2013). This is not a complete comprehensive inventory of all trees. Some errors and/or duplicate data may exist. For more information on Austin's urban forest, visit the U.S. Forest Service's Urban Forest Inventory and Analysis report: https://www.fs.usda.gov/treesearch/pubs/50393


#### 🗒 Columns in this Dataset:
- **GEOMETRY** : Feature's geometry type.
- **SPECIES** : Common species name of the tree.
- **DIAMETER** : Diameter of tree measure at breast height 4.5-feet from the ground. Units are measured in inches.
- **LATITUDE** : Estimated latitude of tree location.
- **LONGTITUDE** : Estimated longitude of tree location.
- **New Georeferenced Column** : Similar to GEOMETRY

### 📖 Reading via from `intake` catalog


In [ ]:
catalog = catalog_init()

In [ ]:
data_trees = catalog.austin_trees.read()
data_trees.head(1)

### ✍️ Random checks for anomoly

In [ ]:
data_trees.loc[data_trees['SPECIES'] =='texas persimmon']

In [ ]:
data_trees.loc[data_trees['DIAMETER'] ==0.00].shape

> **Note**
- 3630 enteries have diameter as 0.00
- We could possibly infer that the trees were *cut/uprooted* due to *industrial/climatic* reasons

### ✅ Data stats and additional info

In [ ]:
data_trees.info()

In [ ]:
data_trees.describe()

### 🛠 Pre-processing

In [ ]:
def process_raw(dataset):
    df_macro = dataset.groupby('SPECIES') \
           .agg({'SPECIES':'count', 'DIAMETER':'mean'}) \
           .rename(columns={'SPECIES':'specie_count','DIAMETER':'mean_diameter'}) \
           .reset_index()\
           .sort_values(by=['specie_count','mean_diameter'], ascending=False)
    return df_macro

#### ⬇️ Function to select subset of dataset


In [ ]:
def select_n_specie(count, data_trees):
    if count > 0:
        df_macro = process_raw(data_trees)
        specie_name_list = df_macro[:int(count)].SPECIES.to_list()
        df_raw_subset = data_trees.query('SPECIES in @specie_name_list')
        df_processed_subset = process_raw(df_raw_subset)
    else:
        print("Enter value greater than 0")

    return df_raw_subset, df_processed_subset

#### ⬇️ Usage of the function 

In [ ]:
a,b = select_n_specie(3, data_trees)

## 📊 Insights

### 🌱  Violin plot, specie overview based on diameter distribution

**About violin plot**

 A violin plot depicts distributions of numeric data for one or more groups using density curves.  
 The width of each curve corresponds with the approximate frequency of data points in each region.   
 Densities are frequently accompanied by an overlaid chart type, such as box plot, to provide additional information.

In [ ]:
key_dimensions   = [('DIAMETER', 'Diameter (inches)')]
value_dimensions = [('SPECIES', 'Specie name')]


count_widget = pn.widgets.IntSlider(name = 'Specie count', value = 5, start = 1, end = 15)
hover = HoverTool(tooltips=[('DIAMETER','$y')])

@pn.depends(count_widget.param.value)
def plotting_specie(count_widget):
    df_top_10_raw,df_top_10_processed = select_n_specie(count_widget, data_trees)
    fig = hv.Violin(df_top_10_raw, value_dimensions, key_dimensions).opts(xrotation=45, width=800, height=400, 
                                                                          violin_fill_color=dim('SPECIES').str(), 
                                                                          cmap='Set1', tools=[hover])
    return fig

widgets = pn.WidgetBox(count_widget)
pn.Column(widgets, plotting_specie)

### Insights

From the max width of the violin plot we can infer the respective concentration of diameter value, for example *Pecan* has maximum concentration (mean) of diameter at 18.4 inch

### 📖 Distribution plot based mean diameter and abundance [Top 10 specie]

In [ ]:
value_dimensions   = [('mean_diameter', 'Mean diameter (measure unit=inches)'), ('specie_count', 'Specie Count')]
key_dimensions = [('SPECIES', 'SPECIES')]
df_top_10_raw,df_top_10_processed = select_n_specie(10, data_trees)
macro = hv.Table(df_top_10_processed, key_dimensions, value_dimensions)

plot_each_specie = macro.to.table('specie_count', 'Mean diameter (measure unit=inches)').opts(height=50, width=400)
hover = HoverTool(tooltips=[('DIAMETER','$x')])
plot_each_specie + hv.Distribution(
    data=df_top_10_raw,
    kdims=['DIAMETER'],
    vdims=['SPECIES'],
).groupby(
    'SPECIES'
).opts(tools=[hover])

### Insight
- The distributions are heavily right skewed (Positive skewed) normal distributions.
- We can infer from above distributions that diameter *Mean  >  Median  >  Mode*

### 🧭 Geo plot

In [ ]:
%%opts Points [width=900 height=650 title="Location of different species in Austin" tools=["hover"]]
%%opts Points (color="dodgerblue" hover_color="lime" line_color="black")


In [ ]:
us_pop_points = gv.Points(data_trees[:5000],
                                kdims=["LONGTITUDE", "LATITUDE"],
                                vdims=["SPECIES"],
                            ).opts(
                                size=12,
                            )

us_pop_points * gv.tile_sources.Wikipedia()